In [1]:
from IPython.display import HTML

HTML("""
<button id="code-show-switch-btn">スクリプトを非表示にする</button>

<script>
var code_show = true;

function switch_display_setting() {
    var switch_btn = $("#code-show-switch-btn");
    if (code_show) {
        $("div.input").hide();
        code_show = false;
        switch_btn.text("スクリプトを表示する");
    }else {
        $("div.input").show();
        code_show = true;
        switch_btn.text("スクリプトを非表示にする");
    }
}

$("#code-show-switch-btn").click(switch_display_setting);
</script>
""")

In [2]:
# For mac-OS environment (Kobayahsi)
import stata_setup
stata_setup.config("/Applications/stata 17", "mp")

import pandas as pd
import numpy as numpy


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 4-core  perpetual
Serial number: 501706353196
  Licensed to: Kobayashi Ryuki
               Keio University

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [6]:
# For windows environment (kakehi)
import numpy as np
import matplotlib.pyplot as plt
import markdown
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.precision = 4
import stata_setup
stata_setup.config("C://Program Files//Stata18", "mp")
#md = markdown.Markdown(extensions=['pymdownx.tabbed'])

ModuleNotFoundError: No module named 'counterfactual'

# Robustness

## Setup

# Merged Commuting Time Single IV

## the 90-120 and above 120 commuting time are merged

In [3]:
%%stata
qui{
use ../data/suicide_analysis_data_static,clear
xtset citycode covid
    
/* Local macro */
local absvars="citycode covid"
local clsvars="citycode"
local absvars_ind "i.citycode i.covid"    
local gmm2="gmm2s"
local log_="log_"
sum suicide
local suicide_t_m=r(mean)
sum suicide_m
local suicide_m_m=r(mean)
sum suicide_f
local suicide_f_m=r(mean)
sum `log_'active_opening_ratio
local log_active_opening_ratio_m=r(mean)
sum `log_'unemployment_rate
local log_unemployment_rate_m=r(mean)
sum `log_'patient
local log_patient_m=r(mean)
sum log_active_opening_weighted
local log_active_opening_weighted_m=r(mean)
sum log_bankruptcy_debt
local log_bankruptcy_debt_m=r(mean)

* bundle commute time IV variable
local commute_time_ratio_ivs = "commuter_ratio_no commuter_ratio_u15 commuter_ratio_15_30 commuter_ratio_30_45 commuter_ratio_45_60 commuter_ratio_60_90 commuter_ratio_90_120 commuter_ratio_o120 "

* For Tables
local txt_u20="Under 20"
local txt_20_29="20-29"
local txt_30_39="30-39"
local txt_40_49="40-49"
local txt_50_59="50-59"
local txt_o60="60 or over"

/* Transformation of log(1+var) */
local paracomp_ct_1st="(D_commuting_time_iv: _b[D1.commuting_time_iv])  "

local paracomp_f_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) "
local paracomp_f_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_f_m')/(`suicide_f_m'))  "
local paracomp_f_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_f_m')/(`suicide_f_m'))  "

local paracomp_m_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) "
local paracomp_m_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_m_m')/(`suicide_m_m'))  "
local paracomp_m_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_m_m')/(`suicide_m_m'))  "
}


. qui{

. 


## Female

In [4]:
%%stata
qui{
* FE
reghdfe D.suicide_f_log D_stay_home, cluster(`clsvars') noabsorb
nlcom `paracomp_f_stayhome', post
est store suicide_f_FE

/* Commuting time shift-share IV */
* IV (1st & 2nd Stage)
ivreg2 D.suicide_f_log (D_stay_home=D_commuting_time_iv2), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_f_ct

** Save 2nd stage result
nlcom `paracomp_f_stayhome', post
est store suicide_f_ct_2nd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_f_ct_1st

** Add 1st stage statistics
est restore tmp_suicide_f_ct
// Fist-stage F-stats
estadd scalar F = `e(widstat)' : suicide_f_ct_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_f_ct_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_f_ct_1st

** Reduced form
reghdfe D.suicide_f_log D_commuting_time_iv2, cluster(`clsvars') noabsorb
nlcom `paracomp_f_ct_reduced', post
est store suicide_f_ct_reduced                                                                                                
                                                                                               
#delimit ;
noi: esttab suicide_f_FE suicide_f_ct_1st suicide_f_ct_2nd suicide_f_ct_reduced, 
drop(_cons) label nomtitles coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) collabels(none)
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1)) 
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) 
labels("Effective F statistic" "tau=10\%" "Observation")) 
varwidth(10) modelwidth(10);
                                                                                                                                                                                              

noi: esttab suicide_f_FE suicide_f_ct_1st suicide_f_ct_2nd suicide_f_ct_reduced 
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_female.tex", 
drop(_cons) label nomtitles coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) collabels(none)
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span}))
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) 
labels("Effective F statistic" "$\tau$=10\%" "Observation"))
booktabs alignment(D{.}{.}{-1})
replace;
};


. qui{

------------------------------------------------------
                  OLS  1st Stage         IV Reduced ~m
                  (1)        (2)        (3)        (4)
------------------------------------------------------
$\Delta$~e      1.109                 0.805           
              (0.432)               (0.554)           
              [0.010]               [0.146]           
$\Delta$~V                 1.701                      
                         (0.038)                      
                         [0.000]                      
$\Delta$~V                                       1.369
                                               (0.946)
                                               [0.148]
------------------------------------------------------
Effectiv~c              1965.789                      
tau=10\%                  23.109                      
Observat~n       1241       1241       1241       1241
------------------------------------------------------
(

## Male

In [5]:
%%stata
qui{
* FE
reghdfe D.suicide_m_log D_stay_home, cluster(`clsvars') noabsorb
nlcom `paracomp_m_stayhome', post
est store suicide_m_FE

/* Commuting time shift-share IV */
* IV (1st & 2nd Stage)
ivreg2 D.suicide_m_log (D_stay_home=D_commuting_time_iv2), gmm2s cluster(`clsvars') first savefirst savefprefix(F_1st_)
est store tmp_suicide_m_ct

** Save 2nd stage result
nlcom `paracomp_m_stayhome', post
est store suicide_m_ct_2nd

** Save 1st stage result
est restore F_1st_D_stay_home
est store suicide_m_ct_1st

** Add 1st stage statistics
est restore tmp_suicide_m_ct
// Fist-stage F-stats
estadd scalar F = `e(widstat)' : suicide_m_ct_1st
weakivtest
estadd scalar F_eff=r(F_eff) : suicide_m_ct_1st
estadd scalar c_TSLS_10=r(c_TSLS_10) : suicide_m_ct_1st

** Reduced form
reghdfe D.suicide_m_log D_commuting_time_iv2, cluster(`clsvars') noabsorb
nlcom `paracomp_m_ct_reduced', post
est store suicide_m_ct_reduced                                                                                                
                                                                                               
#delimit ;
noi: esttab suicide_m_FE suicide_m_ct_1st suicide_m_ct_2nd suicide_m_ct_reduced, 
drop(_cons) label nomtitles coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) collabels(none)
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1)) 
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) 
labels("Effective F statistic" "tau=10\%" "Observation")) 
varwidth(10) modelwidth(10);
                                                                                                                                                                                              

noi: esttab suicide_m_FE suicide_m_ct_1st suicide_m_ct_2nd suicide_m_ct_reduced 
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_male.tex", 
drop(_cons) label nomtitles coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) collabels(none)
mgroups("OLS" "1st Stage" "IV" "Reduced form", pattern(1 1 1 1) prefix(\multicolumn{@span}{c}{) suffix(}) span erepeat(\cmidrule(lr){@span}))
stats(F_eff c_TSLS_10 N, layout(@ @ @) fmt(3 3 %5.0f) 
labels("Effective F statistic" "$\tau$=10\%" "Observation"))
booktabs alignment(D{.}{.}{-1})
replace;
};


. qui{

------------------------------------------------------
                  OLS  1st Stage         IV Reduced ~m
                  (1)        (2)        (3)        (4)
------------------------------------------------------
$\Delta$~e      0.745                 0.999           
              (0.335)               (0.422)           
              [0.026]               [0.018]           
$\Delta$~V                 1.701                      
                         (0.038)                      
                         [0.000]                      
$\Delta$~V                                       1.699
                                               (0.718)
                                               [0.018]
------------------------------------------------------
Effectiv~c              1965.789                      
tau=10\%                  23.109                      
Observat~n       1241       1241       1241       1241
------------------------------------------------------
(

In [6]:
%%stata
qui{
foreach x in u20 20_29 30_39 40_49 50_59 o60 {

sum suicide_f_`x'
local suicide_f_m=r(mean)
sum suicide_m_`x'
local suicide_m_m=r(mean)  

* Define label temporary    
label variable stay_home "Stay-at-home index"
    
#delimit ;
local paracomp_f_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_f_m')/(`suicide_f_m')) ";
local paracomp_f_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_f_m')/(`suicide_f_m'))  ";
local paracomp_f_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_f_m')/(`suicide_f_m'))  ";

local paracomp_m_stayhome="(stay_home: _b[D_stay_home]*(1+`suicide_m_m')/(`suicide_m_m')) ";
local paracomp_m_cr_reduced="(commuter_ratio: _b[commuter_ratio]*(1+`suicide_m_m')/(`suicide_m_m'))  ";
local paracomp_m_ct_reduced="(D_commuting_time_iv: _b[D_commuting_time_iv]*(1+`suicide_m_m')/(`suicide_m_m'))  ";


#delimit cr
/* Female */
/* Commuting time shift-share IV */
ivreg2 D.suicide_f_`x'_log (D_stay_home=D.commuting_time_iv2), gmm2s first cluster(`clsvars')
est store tmp_suicide_f_`x'_ct
nlcom `paracomp_f_stayhome', post
est store suicide_f_`x'_ct_2nd
    
** Reduced form
reghdfe D.suicide_f_`x'_log D_commuting_time_iv, cluster(`clsvars') noabsorb
nlcom `paracomp_f_ct_reduced', post
est store suicide_f_`x'_ct_reduced

/* Male */
/* Commuting time shift-share IV */
ivreg2 D.suicide_m_`x'_log (D_stay_home=D.commuting_time_iv2), gmm2s first cluster(`clsvars')
est store tmp_suicide_m_`x'_ct
nlcom `paracomp_m_stayhome', post
est store suicide_m_`x'_ct_2nd

** Reduced form
reghdfe D.suicide_m_`x'_log D_commuting_time_iv2, cluster(`clsvars') noabsorb
nlcom `paracomp_f_ct_reduced', post
est store suicide_m_`x'_ct_reduced

}

#delimit ;
noi: di "******* Female: Commuing-time SSIV *******"; 
noi:esttab 
suicide_f_u20_ct_2nd suicide_f_20_29_ct_2nd suicide_f_30_39_ct_2nd suicide_f_40_49_ct_2nd suicide_f_50_59_ct_2nd suicide_f_o60_ct_2nd,
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") 
collabels(none)
label coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
stats(N, fmt(%5.0f) labels("Observation"))
varwidth(4) modelwidth(10);

esttab
suicide_f_u20_ct_2nd suicide_f_20_29_ct_2nd suicide_f_30_39_ct_2nd suicide_f_40_49_ct_2nd suicide_f_50_59_ct_2nd suicide_f_o60_ct_2nd
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_female_all_age.tex", 
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60")
collabels(none)
label coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
stats(N, fmt(%5.0f) labels("Observation"))
booktabs alignment(D{.}{.}{-1})
replace;

noi: di "******* Female: Commuing-time Reduced form *******"; 
noi:esttab 
suicide_f_u20_ct_reduced suicide_f_20_29_ct_reduced suicide_f_30_39_ct_reduced suicide_f_40_49_ct_reduced suicide_f_50_59_ct_reduced suicide_f_o60_ct_reduced,
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") 
collabels(none)
label
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
stats(N, fmt(%5.0f) labels("Observation"))
varwidth(4) modelwidth(10);

noi: di "******* Male: Commuing-time SSIV *******"; 
noi:esttab 
suicide_m_u20_ct_2nd suicide_m_20_29_ct_2nd suicide_m_30_39_ct_2nd suicide_m_40_49_ct_2nd suicide_m_50_59_ct_2nd suicide_m_o60_ct_2nd, 
collabels(none)
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) star(* 0.10 ** 0.05  *** 0.01) label
stats(N, fmt(%5.0f) labels("Observation"))
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") varwidth(4) modelwidth(10);

esttab
suicide_m_u20_ct_2nd suicide_m_20_29_ct_2nd suicide_m_30_39_ct_2nd suicide_m_40_49_ct_2nd suicide_m_50_59_ct_2nd suicide_m_o60_ct_2nd
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_male_all_age.tex", 
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60")
collabels(none)
label coeflabels(stay_home "$\Delta$ StayHome")
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
stats(N, fmt(%5.0f) labels("Observation"))
booktabs alignment(D{.}{.}{-1})
replace;

noi: di "******* Male: Commuing-time Reduced form *******"; 
noi:esttab 
suicide_m_u20_ct_reduced suicide_m_20_29_ct_reduced suicide_m_30_39_ct_reduced suicide_m_40_49_ct_reduced suicide_m_50_59_ct_reduced suicide_m_o60_ct_reduced,
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") 
collabels(none)
label
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
stats(N, fmt(%5.0f) labels("Observation"))
varwidth(4) modelwidth(10);
};


. qui{
******* Female: Commuing-time SSIV *******

----------------------------------------------------------------------
            (1)        (2)        (3)        (4)        (5)        (6)
       Under 20      20-29      30-39      40-49      50-59    Over 60
----------------------------------------------------------------------
$\D~e      4.837      1.479      1.812      1.299      1.727      1.214
        (2.023)    (1.183)    (1.287)    (0.990)    (1.045)    (0.771)
        [0.017]    [0.211]    [0.159]    [0.189]    [0.098]    [0.115]
----------------------------------------------------------------------
Obs~n       1241       1241       1241       1241       1241       1241
----------------------------------------------------------------------
******* Female: Commuing-time Reduced form *******

----------------------------------------------------------------------
            (1)        (2)        (3)        (4)        (5)        (6)
       Under 20      20-29      30-39     

#### Export stacked tables of IV & Reduced form

In [7]:
%%stata
qui{
#delimit ;
/* Female */;
/* commuting-time-telework shift-share IV */;
esttab 
suicide_f_u20_ct_2nd suicide_f_20_29_ct_2nd suicide_f_30_39_ct_2nd suicide_f_40_49_ct_2nd suicide_f_50_59_ct_2nd suicide_f_o60_ct_2nd 
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_female_all_age.tex",
prehead("\begin{tabular}{l*{6}{c}} \toprule") 
posthead("\hline \\ \multicolumn{1}{l}{\textit{Panel A: IV}} \\\\[-1ex]") 
fragment
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
collabels(none) nostar
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") 
nomtitles noobs label coeflabels(stay_home "$\Delta$ StayHome")
booktabs alignment(D{.}{.}{-1})
replace;

/* commuting-time-telework shift-share IV */;
esttab 
suicide_f_u20_ct_reduced suicide_f_20_29_ct_reduced suicide_f_30_39_ct_reduced suicide_f_40_49_ct_reduced suicide_f_50_59_ct_reduced suicide_f_o60_ct_reduced
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_female_all_age.tex",
posthead("\hline \\ \multicolumn{1}{l}{\textit{Panel B: Reduced form}} \\\\[-1ex]") 
fragment 
append 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
collabels(none)
stats(N, fmt(%5.0f) labels("Observation"))
nomtitles nonumbers nolines 
booktabs alignment(D{.}{.}{-1}) 
prefoot("\hline") 
postfoot("\bottomrule \end{tabular}") 
label;
};


. qui{

. 


In [8]:
%%stata
qui{
#delimit ;
/* Male */;
/* commuting-time-telework shift-share IV */;
esttab 
suicide_m_u20_ct_2nd suicide_m_20_29_ct_2nd suicide_m_30_39_ct_2nd suicide_m_40_49_ct_2nd suicide_m_50_59_ct_2nd suicide_m_o60_ct_2nd 
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_male_all_age.tex",
prehead("\begin{tabular}{l*{6}{c}} \toprule") 
posthead("\hline \\ \multicolumn{1}{l}{\textit{Panel A: IV}} \\\\[-1ex]") 
fragment
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
collabels(none) nostar
mlabels("Under 20" "20-29" "30-39" "40-49" "50-59" "Over 60") 
nomtitles noobs label coeflabels(stay_home "$\Delta$ StayHome")
booktabs alignment(D{.}{.}{-1})
replace;

/* commuting-time-telework shift-share IV */;
esttab 
suicide_m_u20_ct_reduced suicide_m_20_29_ct_reduced suicide_m_30_39_ct_reduced suicide_m_40_49_ct_reduced suicide_m_50_59_ct_reduced suicide_m_o60_ct_reduced
using "../figuretable/appendix/baseline_stayhome_commuting_time_iv2_male_all_age.tex",
posthead("\hline \\ \multicolumn{1}{l}{\textit{Panel B: Reduced form}} \\\\[-1ex]") 
fragment 
append 
cells(b(fmt(3)) se(fmt(3) par) p(fmt(3) par([ ]))) 
collabels(none)
stats(N, fmt(%5.0f) labels("Observation"))
nomtitles nonumbers nolines 
booktabs alignment(D{.}{.}{-1}) 
prefoot("\hline") 
postfoot("\bottomrule \end{tabular}") 
label;
};


. qui{

. 
